In [5]:
import ta
import sqlalchemy
import pandas as pd

In [6]:
engine = sqlalchemy.create_engine('sqlite:///crypto.db')
#للاتصال بقاعدة البيانات عمل محرك 

In [7]:
names = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"', engine)
#to read names from db 

In [8]:
names = names.name.to_list()
#names to list

In [9]:
#الحصول على السعر والتاريخ من قاعدة البيانات ووضعها في قائمة
framelist = []

for name in names:
    framelist.append(pd.read_sql(f"SELECT Date, Close FROM '{name}'", engine))

In [14]:
def MACDdecision(df):
    df['MACD_diff'] = ta.trend.MACD_diff(df.Close)
    df.loc[(df.MACD_diff) > 0 & (df.MACD_diff.shift(1) < 0), 'Decision MACD'] = 'Buy'
    

In [16]:
def RSI_SMAdecision(df):
    df['RSI'] = ta.momentum.rsi(df.Close, window=200)
    df['SMA200'] = ta.trend.sma_indicator(df.Close, window=200)
    df.loc[(df.Close > df.SMA200) & (df.RSI < 30) , 'Decision RSI/SMA'] = "BUY"

In [17]:
for frame in framelist:
    MACDdecision(frame)
    RSI_SMAdecision(frame)

In [18]:
for name, frame in zip(names,framelist):
    if frame["Decision MACD"].iloc[-1] == "BUY":
        print("Buying Signal MACD for " + name)
    if frame["Decision RSI/SMA"].iloc[-1] == "BUY":
        print("Buying Signal RSI/SMA for " + name)